In [2]:
from utils import load_dataset_tensorflow,plot_accuraccy_loss
import tensorflow as tf
import os
from tensorflow.keras.applications import ConvNeXtLarge,DenseNet121,EfficientNetV2B0,InceptionV3,MobileNetV3Large,ResNet152V2


In [7]:
train_base_dataset, valid_base_dataset = load_dataset_tensorflow(subset="both",validation_split=0.2)
train_aug_dataset, valid_aug_dataset = load_dataset_tensorflow(os.path.join("data","augmented","images"),subset="both",validation_split=0.2)

Found 1650 files belonging to 5 classes.
Using 1320 files for training.
Using 330 files for validation.
Found 1420 files belonging to 5 classes.
Using 1136 files for training.
Using 284 files for validation.


In [63]:

train_ds = train_base_dataset.concatenate(train_aug_dataset)
valid_ds = valid_base_dataset.concatenate(valid_aug_dataset)

def convert_to_rgb(image, label):
    image = tf.image.grayscale_to_rgb(image)  # Convert grayscale image to RGB
    return image, label

train_ds = train_ds.map(convert_to_rgb)
valid_ds = valid_ds.map(convert_to_rgb)

In [71]:
def get_classifier(model_base, input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = model_base(inputs,training=False)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(32,activation="linear")(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(5,activation="softmax")(x)
    model = tf.keras.Model(inputs, outputs)
    return model


In [72]:
base_model = ResNet152V2(include_top=False)
base_model.trainable=False

In [73]:
model = get_classifier(base_model,(256,256,3))

In [74]:
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

In [75]:
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_17 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet152v2 (Functional)        │ (None, 8, 8, 2048)     │    58,331,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 32)             │     4,194,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,526,149 (238.52 MB)

 Trainable params: 4,194,501 (16.00 MB)

 Non-trainable params: 58,331,648 (222.52 MB)

In [77]:
check_callback = tf.keras.callbacks.ModelCheckpoint("models/callbacks/{epoch:02d}-{val_loss:.2f}.keras",verbose=0,save_best_only=True)

history = model.fit(valid_ds,epochs=10,validation_data=valid_ds,callbacks=[check_callback])

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 104s 5s/step - accuracy: 0.3081 - loss: 13157.1963 - val_accuracy: 0.3388 - val_loss: 12010.8584
Epoch 2/10
 8/20 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - accuracy: 0.2151 - loss: 17380.2227

KeyboardInterrupt: 